<a href="https://colab.research.google.com/github/LucaswasTaken/NES-DataScience/blob/main/Aula%2007/Aula_07.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Redes Neurais Convolucionais

In [ ]:
from google.colab import files
uploaded = files.upload()

## Objetivos

* Dados de preparação especificamente para uma CNN
* Criar um modelo CNN mais sofisticado, compreendendo uma maior variedade de camadas de modelos
* Treinar um modelo CNN e observar seu desempenho

## Carregando e preparando os dados

A célula abaixo contém as técnicas de pré-processamento de dados que aprendemos nos laboratórios anteriores. Revise e execute antes de seguir em frente:

In [ ]:
import tensorflow.keras as keras
import pandas as pd

# Load in our data from CSV files
train_df = pd.read_csv("./sign_mnist_train.csv")
valid_df = pd.read_csv("./sign_mnist_valid.csv")

# Separate out our target values
y_train = train_df['label']
y_valid = valid_df['label']
del train_df['label']
del valid_df['label']

# Separate out our image vectors
x_train = train_df.values
x_valid = valid_df.values

# Turn our scalar targets into binary categories
num_classes = 24
y_train = keras.utils.to_categorical(y_train, num_classes)
y_valid = keras.utils.to_categorical(y_valid, num_classes)

# Normalize our image data
x_train = x_train / 255
x_valid = x_valid / 255

## Reformulação de imagens para uma CNN

No último exercício, as imagens individuais em nosso conjunto de dados estão no formato de longas listas de 784 pixels:

In [ ]:
x_train.shape, x_valid.shape

Neste formato, não temos todas as informações sobre quais pixels estão próximos uns dos outros. Por causa disso, não podemos aplicar convoluções que detectem características. Vamos reformular nosso conjunto de dados para que eles estejam em um formato de 28x28 pixels. Isto permitirá que nossas convoluções associem grupos de pixels e detectem características importantes.

Note que para a primeira camada convolucional de nosso modelo, precisamos ter não apenas a altura e largura da imagem, mas também o número de [canais de cor] (https://www.photoshopessentials.com/essentials/rgb/). Nossas imagens são em escala de cinza, portanto, teremos apenas 1 canal.

Isso significa que precisamos converter a forma atual `(27455, 784)` para `(27455, 28, 28, 1)`. Como conveniência, podemos passar o método [reshape](https://numpy.org/doc/stable/reference/generated/numpy.reshape.html#numpy.reshape) um `-1` para qualquer dimensão que desejemos permanecer a mesma, portanto:

In [ ]:
x_train = x_train.reshape(-1,28,28,1)
x_valid = x_valid.reshape(-1,28,28,1)

In [ ]:
x_train.shape

In [ ]:
x_valid.shape

In [ ]:
x_train.shape, x_valid.shape

## Criando um Modelo Convolutivo

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (
    Dense,
    Conv2D,
    MaxPool2D,
    Flatten,
    Dropout,
    BatchNormalization,
)

model = Sequential()
model.add(Conv2D(75, (3, 3), strides=1, padding="same", activation="relu",
                 input_shape=(28, 28, 1)))
model.add(BatchNormalization())
model.add(MaxPool2D((2, 2), strides=2, padding="same"))
model.add(Conv2D(50, (3, 3), strides=1, padding="same", activation="relu"))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(MaxPool2D((2, 2), strides=2, padding="same"))
model.add(Conv2D(25, (3, 3), strides=1, padding="same", activation="relu"))
model.add(BatchNormalization())
model.add(MaxPool2D((2, 2), strides=2, padding="same"))
model.add(Flatten())
model.add(Dense(units=512, activation="relu"))
model.add(Dropout(0.3))
model.add(Dense(units=num_classes, activation="softmax"))

Aqui, resumimos o modelo que acabamos de criar. Observe como ele tem menos parâmetros treináveis do que o modelo do caderno anterior:

In [ ]:
model.summary()

## Compiling the Model

Vamos compilar o modelo exatamente como antes:

In [ ]:
model.compile(loss="categorical_crossentropy", metrics=["accuracy"])

## Treinando o Modelo

Apesar da arquitetura de modelo muito diferente, o treinamento parece exatamente o mesmo. Execute a célula abaixo para treinar durante 20 épocas e vamos ver se a precisão melhora:

In [ ]:
model.fit(x_train, y_train, epochs=10, verbose=1, validation_data=(x_valid, y_valid))

## Discussão dos resultados

Parece que este modelo foi significativamente melhorado! A precisão do treinamento é muito alta, e a precisão da validação também melhorou. Este é um ótimo resultado, pois tudo o que tivemos que fazer foi trocar em um novo modelo.

Você deve ter notado a precisão de validação pulando. Isto é uma indicação de que nosso modelo ainda não está se generalizando perfeitamente. Felizmente, há mais o que podemos fazer. Vamos falar sobre isso na próxima palestra.